## 1. Install 

In [1]:
#!pip install transformers

In [7]:
question="What is the best method to combat the hypercoagulable state seen in COVID-19 ?"
file_name="sm_df.txt"
non_answer='[CLS] what is the best method to combat the hypercoagulable state seen in covid - 19 ? [SEP]'

In [2]:
#from mrjob.job import MRJob
import torch
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
import pandas as pd
import json 

In [ ]:
data_output=[]
with open(file_name, 'r') as f:
    for line in f:
        if len(data_output)>3:
            break
        con=json.loads(line)
        paper_id=con["paper_id"]
        text_dict=con["text_dict"]
        for key in text_dict.keys():
            content = text_dict[key]
            com_id=str(paper_id)+"__"+str(key)

            answer_text=str(content)        

            input_ids = tokenizer.encode(question, answer_text,max_length=500)

            # Report how long the input sequence is.
            #print('Query has {:,} tokens.\n'.format(len(input_ids)))

            # ======== Set Segment IDs ========
            # Search the input_ids for the first instance of the `[SEP]` token.
            sep_index = input_ids.index(tokenizer.sep_token_id)

            # The number of segment A tokens includes the [SEP] token istelf.
            num_seg_a = sep_index + 1

            # The remainder are segment B.
            num_seg_b = len(input_ids) - num_seg_a

            # Construct the list of 0s and 1s.
            segment_ids = [0]*num_seg_a + [1]*num_seg_b

            # There should be a segment_id for every input token.
            assert len(segment_ids) == len(input_ids)

            # ======== Evaluate ========
            # Run our example question through the model.
            start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                            token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

            # ======== Reconstruct Answer ========
            # Find the tokens with the highest `start` and `end` scores.
            answer_start = torch.argmax(start_scores)
            answer_end = torch.argmax(end_scores)


            # Get the string versions of the input tokens.
            tokens = tokenizer.convert_ids_to_tokens(input_ids)

            # Start with the first token.
            answer = tokens[answer_start]

            # Select the remaining answer tokens and join them with whitespace.
            for i in range(answer_start + 1, answer_end + 1):

                # If it's a subword token, then recombine it with the previous token.
                if tokens[i][0:2] == '##':
                    answer += tokens[i][2:]

                # Otherwise, add a space then the token.
                else:
                    answer += ' ' + tokens[i]

            s_scores = start_scores.detach().numpy().flatten()
            e_scores = end_scores.detach().numpy().flatten()  
            print(com_id)
            if (answer!="[CLS]") & (float(max(s_scores))>0)& (len(answer)>0)&(answer!=non_answer):
                data_output.append([paper_id,key,answer,max(s_scores)])

1d7234df1a19a0cf0b4038ceb31e7a97881cdbd7__0_
1d7234df1a19a0cf0b4038ceb31e7a97881cdbd7__1_Purpose
1d7234df1a19a0cf0b4038ceb31e7a97881cdbd7__2_Description
1d7234df1a19a0cf0b4038ceb31e7a97881cdbd7__3_DISCUSSION
0ced1f946cce007aa319a0ba38aef2c4b14dab0e__0_Background


In [6]:
data_output

[['1d7234df1a19a0cf0b4038ceb31e7a97881cdbd7', '0_', 'simulation', 5.4140935],
 ['1d7234df1a19a0cf0b4038ceb31e7a97881cdbd7',
  '1_Purpose',
  'educate',
  3.7028325],
 ['1d7234df1a19a0cf0b4038ceb31e7a97881cdbd7',
  '2_Description',
  'video recording and sharing',
  4.435636],
 ['0ced1f946cce007aa319a0ba38aef2c4b14dab0e',
  '0_Background',
  'neutralizing antibodies',
  1.7667931],
 ['0ced1f946cce007aa319a0ba38aef2c4b14dab0e', '5_Conclusion', 'im', 0.5416325],
 ['0ced1f946cce007aa319a0ba38aef2c4b14dab0e',
  '10_Generation of CTLs in HLA-A2.1/K b transgenic mice',
  '[CLS] what is the best method to combat the hypercoagulable state seen in covid - 19 ? [SEP]',
  1.048755]]